<a href="https://colab.research.google.com/github/PTson2207/Engineer-Selection/blob/main/Exhaustive_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
train = pd.read_csv("/content/drive/MyDrive/HackerEarth/predict the power(KW h) product/dataset/train_process.csv")
test = pd.read_csv("/content/drive/MyDrive/HackerEarth/predict the power(KW h) product/dataset/test_process.csv")
train = train.drop(['Unnamed: 0', 'atmospheric_temperature(°C)'], axis=1)
test = test.drop(['Unnamed: 0', 'atmospheric_temperature(°C)'], axis=1)

In [3]:
X = train.drop(['windmill_generated_power(kW/h)', ], axis=1)
y = train['windmill_generated_power(kW/h)']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# find and remove correlated feature
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [6]:
corr_features = correlation(X_train, 0.8)
print("correlated features:", len(set(corr_features)))

correlated features: 2


In [7]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(22394, 18)

In [8]:
efs1 = EFS(RandomForestRegressor(n_jobs=4),
           min_features=1,
           max_features=4,
           scoring='r2',
           print_progress=True,
           cv=2)
efs1 = efs1.fit(X_train, y_train)

Features: 4047/4047

In [9]:
efs1.best_idx_

(4, 7, 11, 12)

In [10]:
X_train.columns[list(efs1.best_idx_)]

Index(['blades_angle(°)', 'motor_torque(N-m)', 'wind_direction(°)',
       'resistance(ohm)'],
      dtype='object')